# **토큰 역할 : 문자열을 숫자(토큰)로 변환해줘야 함**
- 단어집 큼 --> 모델 훈련에 필요한 메모리 많이 필요
- 단어집 작음 --> 모델 훈련에 필요한 메모리 소모 적음[링크 텍스트](https://)

In [32]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import transformers
from transformers import ElectraTokenizer

In [34]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")

['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]']

In [35]:
# 단어집 크기
tokenizer.vocab_size

35000

In [36]:
tokenizer('안녕하세요')

{'input_ids': [2, 11655, 4279, 8553, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [37]:
# tokenizer가 모든 자연어를 변환해주진 않는다. ex.뷁뷉녊
tokenizer.tokenize("뷃뷁녊. 바람이 울고 있어요.")

['[UNK]', '.', '바람', '##이', '울', '##고', '있', '##어요', '.']

# 토크나이저를 훈련(시킬 때)
1. 어떤 기법을 사용할지?
2. vocab size 결정 : 작으면 메모리 이득, 크면 의미 표현에 유리
3. corpus를 정함 >> 통계적 분석, vocab 결정

tokenizer에 vocab을 직접(수동) 추가하는 방법도 있음

In [38]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='monologg/koelectra-base-v3-discriminator', vocab_size=35000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [39]:
# 문자열을 토큰으로 반환
tokenizer.convert_tokens_to_ids('안녕')

11655

In [40]:
# 넣은 숫자에 맞는 단어 출력. 없으면 UNK(unknown)으로 출력
tokenizer.convert_ids_to_tokens([0,1,2,3, 155,156,30000,35001])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '※', '‼', '킬로그램', '[UNK]']

In [41]:
# 영어 모델 사용해보기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [42]:
# 영어 모델이라 한글은 이상하게 표현됨(ex. ##ㅁㅁ)
print(tokenizer.tokenize('안녕하세요'))
print(tokenizer.tokenize('hello, world'))

['ᄋ', '##ᅡ', '##ᆫ', '##ᄂ', '##ᅧ', '##ᆼ', '##ᄒ', '##ᅡ', '##ᄉ', '##ᅦ', '##ᄋ', '##ᅭ']
['hello', ',', 'world']


In [43]:
tokenizer.vocab_size

30522

In [44]:
# 다국어 모델은 여러 언어에 대한 토큰화도 지원함
multi_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [45]:
# 많이 쪼개서 처리할수록(=의미를 표현하지 못 할수록) 인식이 잘 안 됐다고 봄
# ex. 안녕 --> 안, ##녕 --> 안녕이 vocab에 포함되어 있지 않음
multi_tokenizer.tokenize('안녕하세요')

['안', '##녕', '##하', '##세', '##요']

In [46]:
multi_tokenizer.vocab_size

119547

In [47]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [48]:
tokenizer.convert_ids_to_tokens([11,222,33])

['Ġin', 'Ġdid', 'Ġhave']

# 초거대 언어 모델(GPT-3)
- 다음 token(=단어)을 예측함
- pre-training은 언어 모델을 만드는 과정
- 라벨링 데이터를 pre-training로 돌림
    - ex)오늘 날씨가 너무 좋다. // 긍정
    - '오늘 날씨가 너무 좋다.'는 [긍정/부정] 문장이다.
    - 정답 : 긍정

# **3가지 종류 언어 모델**

**1. Decoder-only 모델(Auto Regressive Model)(GPT 1 ~ 3, ...)**
- few-show learning 사용
- 단점 : 모델이 너무 큼큼
- 점점 더 모델이 커지고 있음(모델이 크고, 데이터를 많이 넣을수록 성능이 좋음)

**2. Encoder-only 모델(Auto Encoder Model)(BERT, ELECTRA, Albert, RoBERTA)**
- input을 vector로 변환해줌. vector는 의미와 관련이 있음.
- pre-training(사전학습+비지도학습) + fine-tuning(지도학습) 사용
- fine-tuning은 downstream task(=추가훈련)(ex. 번역, 문장 분류, 질의응답 등)
- Why?
    1. 성능이 기존 방식보다 훨씬 좋다.
    2. 소량의 데이터로도 학습이 훨씬 빨리 된다.
    
**3. Encoder-Decoder 모델(Transformer 모델과 구조는 동일함)(BART, T5, ...)**
- 생성 관련 task가 사용이 가능함
- pre-training(사전학습+비지도학습) + fine-tuning(지도학습)

---

- Encoder model : 생성 불가능, 추출 가능
- Decoder model : 생성 가능
- embedding : 훈련 됨, 특정 id나 token을 vector로 변환시켜줌
- cf) encoding : 훈련 안 됨, 텍스트를 vector로 변환시켜줌
- decoder에서 추론 시점에 encoder 토큰을 얼마나 참조하냐? -- attention
- self-attention : 자기 자신을 entity로 해서 참조함
- CLS token은 문장 전체의 맥락을 의미하는 token이라고 정의한다. CLS가 embedding 된 것은 문장의 의미를 나타낸다고 본다.
    - CLS token 말고 문장 내 각 token의 평균값을 내는 방법을 사용하기도 함
---
## **한국어 pre-trained model**
- koelectra 모델
- klue-roberta 모델
- korbert 모델

---

**text2image model**
- bert >> 문장을 vector로 만들어준다
- vector >> 이미지로 만든다